In [165]:
import numpy as np
import scipy
import pandas as pnd
import theano
import theano.tensor as T
from lasagne.updates import adam
import time
from collections import Counter

In [2]:
table_csv = pnd.read_csv('data/train.csv')

In [171]:
table = np.int32(table_csv.as_matrix())

table[:, 2] -= 1
users_n = len(np.unique(table[:, 0]))
films_n = len(np.unique(table[:, 1]))
valid_size = int(0.1 * len(table))
valid = table[-valid_size:]
table = table[:-valid_size]

mu = table[:, 2].mean()

#mu_u_sm = np.zeros((users_n, 5), dtype=np.float32)
#counter = Counter(zip(table[:, 0], table[:, 2]))
#for (u,r), c in counter.items():
#    mu_u_sm[u][r] += c/(sum(counter[u,i] for i in range(0, 5)))
#    
#mu_f_sm = np.zeros((films_n, 5), dtype=np.float32)
#counter = Counter(zip(table[:, 1], table[:, 2]))
#for (f,r), c in counter.items():
#    mu_f_sm[f][r] += c/(sum(counter[f,i] for i in range(0, 5)))

print(mu)
print(table.shape)

2.60411121861
(89164901, 3)


In [140]:
print((users_n, films_n))
print(table.shape)

(480189, 17770)
(89164901, 3)


In [141]:
f = 40
U = theano.shared(np.float32(np.random.normal(0, 0.01, (users_n, f))))
F = theano.shared(np.float32(np.random.normal(0, 0.01, (films_n, f))))
bu = theano.shared(np.float32(np.random.normal(0, 0.01, users_n)))
bf = theano.shared(np.float32(np.random.normal(0, 0.01, films_n)))


In [178]:
#f_sm = 40
#U_sm = theano.shared(np.float32(np.random.normal(0, 0.01, (users_n, f_sm))))
#F_sm = theano.shared(np.float32(np.random.normal(0, 0.01, (films_n, f_sm, 5))))
#bu_sm = theano.shared(np.float32(np.random.normal(0, 0.01, users_n)))
#bf_sm = theano.shared(np.float32(np.random.normal(0, 0.01, films_n)))


In [201]:
user_batch, film_batch = T.ivector(), T.ivector()
lmbd = 0.02
ratings = T.ivector()
predict = mu + bu[user_batch] + bf[film_batch] + T.batched_dot(U[user_batch], F[film_batch])
loss = ((predict-ratings)**2).mean()**0.5 +\
        lmbd*(T.mean(U[user_batch]**2) +\
              T.mean(F[film_batch]**2) +\
              T.mean(bu[user_batch]**2) +\
              T.mean(bf[film_batch]**2))
updates = adam(loss, [U, F, bu, bf], learning_rate=0.001)

In [ ]:
#user_batch, film_batch = T.ivector(), T.ivector()
#lmbd = 0.05
#ratings = T.ivector()
#predict = T.nnet.softmax((mu_u_sm[user_batch] + mu_f_sm[film_batch]) / 2 +\
#                         bu_sm[user_batch] + bf_sm[user_batch] +\
#                         T.batched_dot(U_sm[user_batch], F_sm[film_batch]))
#
#loss = (T.nnet.categorical_crossentropy(predict, ratings).mean() +\
#        lmbd*(T.mean(U_sm**2) + T.mean(F_sm**2) + T.mean(bu_sm**2) + T.mean(bf_sm**2)))
#    
#updates = adam(loss, [U_sm, F_sm, bu_sm, bf_sm], learning_rate=0.005)

In [205]:
predict_fn = theano.function([user_batch, film_batch], predict)
train_fn = theano.function([user_batch, film_batch, ratings], [loss, predict], updates=updates)

In [ ]:
start = time.time()
batch_size = 1024*1024
idxs = np.arange(0, len(table), dtype=np.int32)
np.random.shuffle(idxs)  
table = table[idxs]
print(time.time() - start)
start = time.time()
train_fn(table[:batch_size, 0], table[:batch_size, 1], table[:batch_size, 2])
print(time.time() - start)

16.431591033935547
2.813370704650879


In [ ]:
epochs = 100
#best_valid = 100.
#best_model = (U.get_value(), F.get_value(), bu.get_value(), bf.get_value())
#U.set_value(best_model[0])
#F.set_value(best_model[1])
#bu.set_value(best_model[2])
#bf.set_value(best_model[3])

for epoch in range(epochs):
    idxs = np.arange(0, len(table), dtype=np.int32)
    np.random.shuffle(idxs)
    table = table[idxs]
    res = 0
    cnt = 0
    start = time.time()
    train_rmse = 0
    for i in range(0, len(table), batch_size):
        if i + batch_size > len(table):
            break
        idx = np.arange(i, i+batch_size)
        tmp = train_fn(table[idx, 0], table[idx, 1], table[idx, 2])
        res += tmp[0]
        train_rmse += ((table[idx, 2] - tmp[1])**2).sum()
        cnt += 1
    print('epoch', epoch)
    print('loss ', res/cnt)
    print('train rmse:', (train_rmse/(cnt*batch_size))**0.5)
    
    valid_pred = predict_fn(valid[:, 0], valid[:, 1])
    valid_true = valid[:, 2]
    valid_scor = ((valid_pred-valid_true)**2).mean()**0.5
    if valid_scor < best_valid:
        best_valid = valid_scor
        best_model = (U.get_value(), F.get_value(), bu.get_value(), bf.get_value())
    print('valid rmse:', valid_scor)
    print('time:', time.time()-start)


epoch 0
loss  0.736817429351
train rmse: 0.729591452343
valid rmse: 0.866191486417
time: 245.78619408607483


KeyboardInterrupt: 

In [147]:
test_table = pnd.read_csv('data/test-ids.csv')
test = np.int32(test_table.as_matrix())

In [209]:
best_valid

0.85994188285434692

In [ ]:
U.set_value(best_model[0])
F.set_value(best_model[1])
bu.set_value(best_model[2])
bf.set_value(best_model[3])

predictions = []
for i in range(0, len(test), batch_size):
    predictions += list(predict_fn(test[:, 1], test[:, 2]))
    print(min((i+batch_size), len(test))/len(test))
    
#predictions = np.array(predictions)
with open('s.txt', 'w+') as out:
    out.write('Id,Prediction\n')
    for i, p in zip(list(test[:, 0]), predictions):
        out.write(str(i) + ',' + str(p) + '\n')

0.7445184057029456
1.0
